In [1]:
import importlib
import GUI_notebook
importlib.reload(GUI_notebook)
vb = GUI_notebook.init(In)
vb

Box(children=(Box(children=(Btn(children=['Copy cells'], color='blue lighten-1', height='35px', width='250px')…

```python

class Component1(om.ExplicitComponent):

	def setup(self):
		self.add_input('x_name', val=np.nan)
		self.add_output('y_name')
		self.add_output('a_name')

	def setup_partials(self):
		self.declare_partials('*', '*', method='fd')

	def compute(self, inputs, outputs, discrete_inputs=None, discrete_outputs=None):

		x = inputs['x_name']

		y = x + 1
		a = np.pi**2*r + 1.54

		outputs['y_name'] = y
		outputs['a_name'] = a


class Component 2(om.ExplicitComponent):

	def setup(self):
		self.add_input('v_name', val=0, units='kg')
		self.add_input('w_name', val=np.nan)
		self.add_output('u_name')

	def setup_partials(self):
		self.declare_partials('*', '*', method='fd')

	def compute(self, inputs, outputs, discrete_inputs=None, discrete_outputs=None):

		v = inputs['v_name']
		w = inputs['w_name']

		u = np.sin(v + w)**2

		outputs['u_name'] = u


```

In [ ]:
"""
---Test Text---

# Component1

x = 2*y + 3*z #[m]
a = b + c*1.2e-3 #[kg]

# Component2

long_variable = a + b #[long_unit]
function_test = np.log(x + np.sin(y)) #[rad]


"""

In [ ]:
"""
---Test Text---
## Group1

# Component1

x = 2*y + 3*z #[m]
a = b + c*1.2e-3 #[kg]

# Component2

long_variable = a + b
function_test = np.log(x + np.sin(y)) #[rad]

## Group2

# Component3

x = 2*y + 3*z
a = b + c

# Component4

long_variable = a + b
function_test = np.log(x + np.sin(y))

"""

In [24]:
# Init
x = 0
z = 1

In [25]:
# Exclude
"""
some commentary
"""

'\nsome commentary\n'

In [26]:
### HG1

## Group1

# Component1
y = x + 1
y = y**2 + z

In [27]:
# Exclude
y

2

In [28]:
# Init
b = 3
c = 4

In [29]:
### HG2
## Group2

# Component2

x = 2*y + 3*z #[m]
a = b + c*1.2e-3 #[kg]

In [30]:
# Exclude
import numpy as np
a = 0
b = 1

In [31]:
# Component3

long_variable = a + b
function_test = np.log(x + np.sin(y)) #[rad]